In [ ]:
# ! pip install mlflow

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [4]:
x,y = make_classification(
    n_features=10,
    n_samples=1000,
    n_informative=2,
    n_redundant=8,
    weights=[0.9,0.1],
    flip_y=0,
    random_state=42
)

np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100]))

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [8]:
params = {
    "solver":"lbfgs",
    "max_iter":1000,
    "multi_class":"auto",
    "random_state":8888
}

lr = LogisticRegression(**params)
lr.fit(x_train,y_train)

y_pred = lr.predict(x_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       270
           1       0.82      0.60      0.69        30

    accuracy                           0.95       300
   macro avg       0.89      0.79      0.83       300
weighted avg       0.94      0.95      0.94       300



In [9]:
report_dict = classification_report(y_test,y_pred,output_dict=True)
report_dict

{'0': {'precision': 0.9568345323741008,
  'recall': 0.9851851851851852,
  'f1-score': 0.9708029197080292,
  'support': 270.0},
 '1': {'precision': 0.8181818181818182,
  'recall': 0.6,
  'f1-score': 0.6923076923076923,
  'support': 30.0},
 'accuracy': 0.9466666666666667,
 'macro avg': {'precision': 0.8875081752779594,
  'recall': 0.7925925925925925,
  'f1-score': 0.8315553060078608,
  'support': 300.0},
 'weighted avg': {'precision': 0.9429692609548727,
  'recall': 0.9466666666666667,
  'f1-score': 0.9429533969679956,
  'support': 300.0}}

In [10]:
import mlflow

In [ ]:
mlflow.set_experiment("1st exp")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        "accuracy":report_dict['accuracy'],
        "recall_class_0":report_dict['0']['recall'],
        "recall_class_1":report_dict['1']['recall'],
        "f1_score_macro":report_dict['macro avg']['f1-score'],
        "f1_score_weighted":report_dict['weighted avg']['f1-score'],
        "Precision_class_0":report_dict['0']['precision'],
        "Precision_class_1":report_dict['1']['precision'],
    })
    
    mlflow.sklearn.log_model(lr,"Logistic Regression")

2025/09/15 17:42:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/15 17:42:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run unruly-colt-555 at: http://127.0.0.1:5000/#/experiments/661059236851633903/runs/5dfdf9e84c6041f5825b4ec56078700d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/661059236851633903


TypeError: 'module' object is not callable

In [15]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(x_train, y_train)
y_pred_log_reg = log_reg.predict(x_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       270
           1       0.78      0.60      0.68        30

    accuracy                           0.94       300
   macro avg       0.87      0.79      0.82       300
weighted avg       0.94      0.94      0.94       300



In [16]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(x_train, y_train)
y_pred_rf = rf_clf.predict(x_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       270
           1       1.00      0.57      0.72        30

    accuracy                           0.96       300
   macro avg       0.98      0.78      0.85       300
weighted avg       0.96      0.96      0.95       300



In [17]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(x_train, y_train)
y_pred_xgb = xgb_clf.predict(x_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.88      0.77      0.82        30

    accuracy                           0.97       300
   macro avg       0.93      0.88      0.90       300
weighted avg       0.97      0.97      0.97       300



[17:46:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



In [18]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(x_train, y_train)

np.unique(y_train_res, return_counts=True)

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(x_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       270
           1       0.76      0.83      0.79        30

    accuracy                           0.96       300
   macro avg       0.87      0.90      0.88       300
weighted avg       0.96      0.96      0.96       300



[17:46:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



In [19]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(C=1, solver='liblinear'), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (x_test, y_test)
    )
]

In [27]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

[17:52:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:52:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



In [21]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [29]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection1")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 403 != 200. Response body: ''